### Goals

0. We need to map velocities to the atmosphere somehow. what parameterizations are "simple"?

  * fixed/solid body rotation
  * differential rotation

1. $K_p-v_{\rm sys}$ diagram for a planet (for now we'll use a stellar spectrum) that is orbiting a star, with a velocity field in its atmosphere

2. estimate the effect of a velocity field throughout the terminator on the transmission spectrum

3. compare GCM velocity fields with observations (in transit transmission and in full-orbit CCFs)

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from expecto import get_spectrum  # downloads phoenix model spectra
from tqdm.auto import tqdm
from ipywidgets import interactive

Set up the lon ($\phi$), lat ($\theta$), and orbital phase ($\xi$) grids:

In [ ]:
n_obs_per_orbit = 50

n_lats = 7
n_lons = 25
phi = np.linspace(0, 2*np.pi, n_lons)

lat_frac = 0.9
theta = np.arcsin(np.linspace(-lat_frac, lat_frac, n_lats))

xi = np.linspace(-np.pi, np.pi, n_obs_per_orbit)

Rp = 1 * u.R_jup

phi2d, theta2d = np.meshgrid(
    phi, theta
)

phi3d, theta3d, xi3d = np.meshgrid(
    phi, theta, xi
)

$$ v(\phi, \theta) = \sin(\theta) $$

In [ ]:
x = Rp * np.cos(theta3d) * np.sin(phi3d)
y = Rp * np.sin(theta3d)
z = Rp * np.sin(theta3d) * np.sin(phi3d)

dx = - np.sin(phi3d - delta_phi)
dy = np.cos(phi3d)
dz = np.zeros_like(phi3d)

plt.scatter(x[..., 0].value.ravel(), y[..., 0].value.ravel(), c=-dx[..., 0].ravel())
plt.colorbar()
plt.gca().set(
    xlabel='x', 
    ylabel='y'
)
plt.gca().set_aspect('equal')

In [ ]:
spec = get_spectrum(2300, 5.0, cache=True)

In [ ]:
minimum_wavelength, maximum_wavelength = 0.5*u.um, 0.501*u.um

mask = (
    (spec.wavelength > minimum_wavelength) & 
    (spec.wavelength < maximum_wavelength)
)

wavelength = spec.wavelength[mask]
flux_cube = np.squeeze(spec.flux[mask][:, None, None, None] * np.ones_like(x)[None, None, None, :])

print(flux_cube.shape)

The shape of the resulting `flux_cube` is (wavelength, lat, lon, time/phase): $N_\lambda, N_\theta, N_\phi$, $N_\xi$

Now let's start with a fixed velocity map:

In [ ]:
eq_velocity = 100 * u.km/u.s
pole_velocity = 5 * u.km/u.s
alpha = float((eq_velocity - pole_velocity) / pole_velocity)

# this is the variation of wind velocity with latitude, written in
# the form often used to describe stellar differential rotation:
differential_rotation = pole_velocity * (1 + alpha * np.cos(theta3d))

# the velocity map will be asymmetric using this longitudinal scale factor:
delta_phi = 0.4  # phase offset of the wind asymmetry
asymmetry = 0.4  # asymmetry strength in units of fraction of differential rotation amplitude:
velocity_map = differential_rotation * (1 + asymmetry * np.sin(phi3d - delta_phi))

radial_velocity_map = velocity_map * -dx

Compute the "visibility" of each spatial pixel on the sphere. `1` is a pixel viewed face-on, `0` is a pixel that is not visible.

In [ ]:
visibility = np.cos(phi3d - np.pi - xi) * np.sin(theta3d + np.pi/2)**2
visibility[visibility < 0] = 0

Plot them all:

In [ ]:
fig = plt.figure(figsize=(25, 3))

viz_n_phases = 8
for i in range(viz_n_phases):
    axis = fig.add_subplot(100 + 10 * viz_n_phases + 1 + i, projection='mollweide')
    cax = axis.pcolormesh(phi - np.pi, theta, visibility[..., int(i*len(xi) / viz_n_phases)])
    #plt.colorbar(cax, ax=axis, label='visibility')
fig.suptitle('Visibility over time, as the world turns')

In [ ]:
xi_ind = len(xi)//2
fig = plt.figure(figsize=(20, 4))
ax0 = fig.add_subplot(131, projection='mollweide')
ax1 = fig.add_subplot(132, projection='mollweide')
ax2 = fig.add_subplot(133, projection='mollweide')
cax = ax0.pcolormesh(phi - np.pi, theta, velocity_map[..., xi_ind])
plt.colorbar(cax, ax=ax0, label='local velocity [km/s]')
cax = ax1.pcolormesh(phi - np.pi, theta, visibility[..., xi_ind])
plt.colorbar(cax, ax=ax1, label='fractional visibility')
cax = ax2.pcolormesh(phi - np.pi, theta, radial_velocity_map[..., xi_ind])
plt.colorbar(cax, ax=ax2, label='radial velocity [km/s]')

cax = ax2.contour(phi - np.pi, theta, visibility[..., xi_ind], levels=[0.1, 0.3, 0.8], cmap=plt.cm.Greys_r)

ax0.set_title('local velocity')
ax1.set_title('visibility')
ax2.set_title('map=radial velocity;\ncontour=visibility')
plt.show()

Now for each spatial pixel on the sphere in $\theta, \phi$, compute the redshifts/blueshifts for each wavelength at each pixel:

In [ ]:
wavelength_cube = radial_velocity_map[None, ...].to(u.nm, u.doppler_optical(wavelength[:, None, None, None]))

Take the flux cube (wave, lat, lon) and scale it by the visibility of each spatial pixel on the sphere:

In [ ]:
weighted_flux_cube = flux_cube * visibility[None, ...] # weighting the spectrum by the visibility

The weighted flux cube has doppler shifted wavelengths, but the observer only sees a single set of wavelengths (named here `wavelength`). Let's interpolate the doppler shifted spectra onto each observer-frame wavelength, at each spatial pixel:

In [ ]:
shifted_weighted_flux_cube = np.empty(weighted_flux_cube.shape)
new_shape = (weighted_flux_cube.shape[0], np.prod(weighted_flux_cube.shape[1:-1]))

for j in tqdm(range(len(xi))):
    # We're going to take the 3D cube, and make it 2D temporarily so we can interpolate efficiently:
    wavelength_cube_j = wavelength_cube[..., j].reshape(new_shape)
    shifted_weighted_flux_cube_j = np.empty(new_shape)
    weighted_flux_cube_j = weighted_flux_cube[..., j].reshape(new_shape)
    # loop over the (now single) spatial dimension and interpolate in the wavelength dimension
    for i in range(shifted_weighted_flux_cube_j.shape[1]):
        shifted_weighted_flux_cube_j[:, i] = np.interp(
            wavelength.to(u.um).value, 
            wavelength_cube_j[:, i].to(u.um).value, 
            weighted_flux_cube_j[:, i].value
        )
    shifted_weighted_flux_cube[..., j] = shifted_weighted_flux_cube_j.reshape(weighted_flux_cube.shape[:-1])

In [ ]:
# the observed spectrum of the visible hemisphere, as a function of time, is: 
observed_spectrum = shifted_weighted_flux_cube.sum(axis=(1, 2))

In [ ]:
# this normalization factor shifts a single spatial element's spectrum up to the same scale
# as the observed spectrum of a full hemisphere:
norm = spec.flux[mask].max() / observed_spectrum.max()
plt.plot(wavelength, spec.flux[mask] / norm, label='intrinsic local spectrum')
plt.plot(wavelength, observed_spectrum[:, 0], label=f'full-disk spectrum at $\\xi = {xi[0]:.1f}$')
plt.plot(wavelength, observed_spectrum[:, observed_spectrum.shape[1]//2], label=f'full-disk spectrum at $\\xi = {xi[observed_spectrum.shape[1]//2]:.1f}$')
plt.legend()

In [ ]:
def spec_interact(phase):
    ind = int(phase/360 * len(xi))
    fig = plt.figure(figsize=(25, 3))
    ax0 = fig.add_subplot(141, projection='mollweide')
    ax1 = fig.add_subplot(142, projection='mollweide')
    ax2 = fig.add_subplot(143)

    cax = ax0.pcolormesh(phi - np.pi, theta, visibility[..., ind])
    plt.colorbar(cax, ax=ax0, label='fractional visibility')
    cax = ax1.pcolormesh(phi - np.pi, theta, radial_velocity_map[..., ind])
    plt.colorbar(cax, ax=ax1, label='radial velocity [km/s]')

    cax = ax1.contour(phi - np.pi, theta, visibility[..., ind], levels=[0.1, 0.3, 0.8], cmap=plt.cm.Greys_r)

    ax0.set_title('visibility')
    ax1.set_title('map=radial velocity;\ncontour=visibility')
    
    norm = spec.flux[mask].max() / observed_spectrum.max()
    ax2.plot(wavelength, spec.flux[mask] / norm, label='intrinsic local spectrum')
    ax2.plot(wavelength, observed_spectrum[:, ind], label=f'full-disk spectrum at $\\xi = {xi[ind]:.1f}$')
    ax2.legend()
    ax2.set(
        xlabel='Wavelength [Angstrom]',
        ylabel='Flux'
    )
    
print(
    'Vary orbital phase of the planet (degrees) to see the \nvisible hemisphere change'
    ', and see the evolution\nof the observed spectrum on the right:\n\n'
)
step = int(360/len(xi))
interactive_plot = interactive(spec_interact, phase=(0, 360 - step, step))
output = interactive_plot.children[-1]
interactive_plot